In [ ]:
import tweepy
import json
from pymongo import MongoClient
from bson.objectid import ObjectId
from bson.json_util import dumps
import time

In [ ]:
CONSUMER_KEY = ''
CONSUMER_SECRET = ''
ACCESS_TOKEN = ''
ACCESS_TOKEN_SECRET = ''

In [ ]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
MONGO_HOST = 'mongodb://localhost:27017/'

client = MongoClient(MONGO_HOST)

db = client['8M2020_sample']

collection = db.tweets_sample

collection2 = db.tweets_sample_updated

In [ ]:
num_docs = collection.count_documents({})
num_docs

In [ ]:
num_limit = 1000
num_skip = 0

In [ ]:
print('Atualização de tweets iniciada')
print(time.asctime(time.localtime(time.time())))

In [ ]:
print('\nExtraindo 5000 tweets para atualização\n')
docs = list(collection.find({}, {'id': True}).limit(num_limit).skip(num_skip))

In [ ]:
tweets_ids = [doc['id'] for doc in docs]
# tweets_ids
# len(tweets_ids)

In [ ]:
start = 0
end = 100

In [ ]:
while end <= len(tweets_ids):
    updated_tweets = []
    tweets = []

    updated_tweets = api.statuses_lookup(
        tweets_ids[start:end], tweet_mode='extended')
    # tweets_ids_updated = [ut._json['id'] for ut in updated_tweets]
    # len(tweets_ids_updated)
    # set(tweets_ids) - set(tweets_ids_updated)
    tweets = [ut._json for ut in updated_tweets]
    collection2.insert_many(tweets)

    num_skip = num_skip + 100

    print(f'{len(tweets_ids[start:end])} tweets enviados | {len(tweets)} atualizados | Total enviado: {num_skip}/{num_docs} | {(num_skip*100)/num_docs:.2f}%')

    start = start + 100
    end = end + 100

In [ ]:
while num_skip < num_docs:
    docs = []
    tweets_ids = []

    print('\nExtraindo 5000 tweets para atualização\n')
    docs = list(collection.find({}, {'id': True}
                                ).limit(num_limit).skip(num_skip))
    tweets_ids = [doc['id'] for doc in docs]

    start = 0
    end = 100

    while end <= len(tweets_ids):
        updated_tweets = []
        tweets = []

        updated_tweets = api.statuses_lookup(
            tweets_ids[start:end], tweet_mode='extended')
        tweets = [ut._json for ut in updated_tweets]
        collection2.insert_many(tweets)

        num_skip = num_skip + 100

        print(
            f'{len(tweets_ids[start:end])} tweets enviados | {len(tweets)} atualizados | Total enviado: {num_skip} | {(num_skip*100)/num_docs:.2f}%')

        start = start + 100
        end = end + 100

print('Atualização de tweets finalizada')
print(time.asctime(time.localtime(time.time())))

In [ ]:
print('Diferença entre a base original e a atualizada: ')
print(collection.count_documents({}) - collection2.count_documents({}))